In [1]:
# 환경 구축
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import parameter
from torch import Tensor
import math

import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

#재현성 보장을 위한 시드 설정 (난수 생성의 초기값을 고정)
torch.manual_seed(100)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(100)

In [1]:
# 로그데이터

## 데이터 구성 방식
# 타임스탬프:N, 각 시점 x,y,z & 속도 (vx, vy, vz) => [x, y, z, (xy, xz, yz)]
# takeoff~landing까지 한 세션
# 각 비행마다 하나의 시퀀스로 분리 -> 시퀀스 길이가 다 다르면 패딩 적용
# 입출력 슬라이딩 윈도우 (e.g. 10개의 timestep 입력 -> 3개 timestep 예측)


##1. ulog2csv
# 여러 개의 csv가 나오면 ‘vehicle_local_position_0’, ‘vehicle_attitude_0’ 등 주요 토픽 파일 선택
#### 어 여기서 좋은 생각: 궤적 예측할 때 위치만 넣는 게 아니라 attitude 값도 넣으면 더 좋을 것 같은데????????????????
# 현재 위치 + 현재 자세값... 와 이거지

##2. csv -> dataframe 로딩
# 또는 주요 위치, 속도, 타임스탬프 등만 사용
##3. LSTM input 데이터 구조로 전처리
# 필요 컬럼만 추출: timestamp, x, y, (z), (vx, vy, vz)
# 시계열 정렬/슬라이딩 윈도우/정규화 등 단계 적용


In [25]:
#1. ulog2csv
import sys
sys.path.insert(0,"C:/workspace_data")

import convert2csv_1
ulog_file = "C:/workspace_TrajectoryPrediction/d_flightlog/SLMD-030(6).ulg"
convert2csv_1.convert_ulog_to_csv(ulog_file)

Created directory: C:/workspace_TrajectoryPrediction/d_flightlog\SLMD-030(6)
Running command: ulog2csv C:/workspace_TrajectoryPrediction/d_flightlog/SLMD-030(6).ulg -d , -o C:/workspace_TrajectoryPrediction/d_flightlog\SLMD-030(6)
Conversion successful


In [34]:
#2. input data를 dataframe화
# vehicle_id 식별자 추가하자
import pandas as pd

df = pd.read_csv("C:\workspace_TrajectoryPrediction\d_flightlog\SLMD-027(1)\SLMD-027(1)_vehicle_local_position_0.csv")

key_colums = ['timestamp', 'x', 'y', 'z', 'vx', 'vy', 'vz']
print(df[key_colums].isnull().sum())

df_traj = df[key_colums].dropna().reset_index(drop=True)
df_traj.head()



timestamp    0
x            0
y            0
z            0
vx           0
vy           0
vz           0
dtype: int64


<>:5: SyntaxWarning: invalid escape sequence '\w'
<>:5: SyntaxWarning: invalid escape sequence '\w'
C:\Users\admin\AppData\Local\Temp\ipykernel_34072\2267319896.py:5: SyntaxWarning: invalid escape sequence '\w'
  df = pd.read_csv("C:\workspace_TrajectoryPrediction\d_flightlog\SLMD-027(1)\SLMD-027(1)_vehicle_local_position_0.csv")


,timestamp,x,y,z,vx,vy,vz
0,403981202,-0.895930,-1.680361,-1.058567,-0.007286,-0.004946,-0.062832
1,403991203,-0.896126,-1.680820,-1.059796,-0.007379,-0.005119,-0.062848
2,404081259,-0.897799,-1.684500,-1.067588,-0.007970,-0.006045,-0.062162
3,404181209,-0.900445,-1.689484,-1.078160,-0.009410,-0.007995,-0.061388
4,404281204,-0.902892,-1.694061,-1.088035,-0.011065,-0.009310,-0.060172


In [35]:
# drone id 추가
df_traj['vehicle_id'] = 'PAF02_20'
df_traj

,timestamp,x,y,z,vx,vy,vz,vehicle_id
0,403981202,-0.895930,-1.680361,-1.058567,-0.007286,-0.004946,-0.062832,PAF02_20
1,403991203,-0.896126,-1.680820,-1.059796,-0.007379,-0.005119,-0.062848,PAF02_20
2,404081259,-0.897799,-1.684500,-1.067588,-0.007970,-0.006045,-0.062162,PAF02_20
3,404181209,-0.900445,-1.689484,-1.078160,-0.009410,-0.007995,-0.061388,PAF02_20
4,404281204,-0.902892,-1.694061,-1.088035,-0.011065,-0.009310,-0.060172,PAF02_20
...,...,...,...,...,...,...,...,...
2548,658981220,642.327300,-39.585392,62.780945,-10.208615,25.539358,27.053604,PAF02_20
2549,659081395,641.311800,-37.054043,65.493200,-10.050382,25.239956,27.319294,PAF02_20
2550,659181249,640.310670,-34.561424,68.244280,-9.927754,24.858090,27.626513,PAF02_20
2551,659281204,639.321900,-32.117190,71.034680,-9.837167,24.505362,27.876990,PAF02_20
